In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [14]:
import pandas as pd 
import duckdb as db
from autogluon.tabular import TabularPredictor
from sklearn.metrics import root_mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

In [3]:
sales = pd.read_csv('data/sales.csv', parse_dates=['date'])
online = pd.read_csv('data/online.csv', parse_dates=['date'])
test = pd.read_csv('data/test.csv', parse_dates=['date'], sep=';')

/tmp/ipykernel_3781/1861528378.py:3: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  test = pd.read_csv('data/test.csv', parse_dates=['date'], sep=';')


In [ ]:
con = db.connect("data/mydb.duckdb")

In [ ]:
combined_sales = db.sql(f"""
    with date_dim as (
        select distinct date 
        from (
            select date from sales 
            union 
            select date from online
        )
    ),
    store_item as (
        select distinct 
            store_id,
            item_id
        from (
            select store_id, item_id from sales
            union
            select store_id, item_id from online
        )
    ),
    date_store_item_cross as (
        select 
            d.date,
            si.store_id,
            si.item_id
        from date_dim d
        cross join store_item si
    ),
    daily_sales as (
        select 
            dsi.date,
            dsi.item_id,
            extract(month from dsi.date) as month,
            extract(year from dsi.date) as year,
            extract(day from dsi.date) as day,
            extract(week from dsi.date) as week,
            case 
                when extract(dow from dsi.date) in (0, 6) then 'weekend'
                else 'weekday'
            end as day_type,
            1 as sample_weight,
            extract(dow from dsi.date) as day_of_week,
            dsi.store_id,
            coalesce(a.quantity, 0) + coalesce(b.quantity, 0) as quantity
        from date_store_item_cross dsi
        left join sales as a 
            on dsi.date = a.date
            and dsi.item_id = a.item_id
            and dsi.store_id = a.store_id
        left join online as b 
            on dsi.date = b.date
            and dsi.item_id = b.item_id
            and dsi.store_id = b.store_id
    )
    
    select 
        *,
        coalesce(avg(quantity) over (
            partition by store_id, item_id
            order by date 
            rows between 7 preceding and 1 preceding
        ), 0) as rolling_7day_avg,
        coalesce(avg(quantity) over (
            partition by store_id, item_id
            order by date 
            rows between 14 preceding and 1 preceding
        ), 0) as rolling_14day_avg,
        coalesce(avg(quantity) over (
            partition by store_id, item_id
            order by date 
            rows between 30 preceding and 1 preceding
        ), 0) as rolling_30day_avg
    from daily_sales
    order by date
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [1]:
import duckdb
import pandas as pd

# Cargar datos originales en DataFrames
sales = pd.read_csv('data/sales.csv', parse_dates=['date'])
online = pd.read_csv('data/online.csv', parse_dates=['date'])
test = pd.read_csv('data/test.csv', parse_dates=['date'], sep=';')

# Abrir conexión a un archivo DuckDB (persistente en disco)
con = duckdb.connect("data/mydb.duckdb")

# Registrar DataFrames como tablas temporales en DuckDB
con.register("sales", sales)
con.register("online", online)

# Crear tabla persistente con tu query
con.execute("""
    CREATE OR REPLACE TABLE combined_sales AS
    with date_dim as (
        select distinct date 
        from (
            select date from sales 
            union 
            select date from online
        )
    ),
    store_item as (
        select distinct 
            store_id,
            item_id
        from (
            select store_id, item_id from sales
            union
            select store_id, item_id from online
        )
    ),
    date_store_item_cross as (
        select 
            d.date,
            si.store_id,
            si.item_id
        from date_dim d
        cross join store_item si
    ),
    daily_sales as (
        select 
            dsi.date,
            dsi.item_id,
            extract(month from dsi.date) as month,
            extract(year from dsi.date) as year,
            extract(day from dsi.date) as day,
            extract(week from dsi.date) as week,
            case 
                when extract(dow from dsi.date) in (0, 6) then 'weekend'
                else 'weekday'
            end as day_type,
            1 as sample_weight,
            extract(dow from dsi.date) as day_of_week,
            dsi.store_id,
            coalesce(a.quantity, 0) + coalesce(b.quantity, 0) as quantity
        from date_store_item_cross dsi
        left join sales as a 
            on dsi.date = a.date
            and dsi.item_id = a.item_id
            and dsi.store_id = a.store_id
        left join online as b 
            on dsi.date = b.date
            and dsi.item_id = b.item_id
            and dsi.store_id = b.store_id
    )
    
    select 
        *,
        coalesce(avg(quantity) over (
            partition by store_id, item_id
            order by date 
            rows between 7 preceding and 1 preceding
        ), 0) as rolling_7day_avg,
        coalesce(avg(quantity) over (
            partition by store_id, item_id
            order by date 
            rows between 14 preceding and 1 preceding
        ), 0) as rolling_14day_avg,
        coalesce(avg(quantity) over (
            partition by store_id, item_id
            order by date 
            rows between 30 preceding and 1 preceding
        ), 0) as rolling_30day_avg
    from daily_sales
    order by date
""")

# Ahora la tabla queda guardada en "data/mydb.duckdb"
# Puedes consultar después sin volver a calcular todo
df = con.execute("SELECT * FROM combined_sales LIMIT 10").df()
print(df.head())

# Exportar también a Parquet si quieres
con.execute("COPY combined_sales TO 'data/combined_sales.parquet' (FORMAT 'parquet')")


/tmp/ipykernel_1227/1470816164.py:7: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  test = pd.read_csv('data/test.csv', parse_dates=['date'], sep=';')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

        date       item_id  month  year  day  week day_type  sample_weight  \
0 2022-08-28  00c76368f791      8  2022   28    34  weekend              1   
1 2022-08-28  020b151fb4fd      8  2022   28    34  weekend              1   
2 2022-08-28  05db904338e0      8  2022   28    34  weekend              1   
3 2022-08-28  0af42b55f108      8  2022   28    34  weekend              1   
4 2022-08-28  0e71cef8e293      8  2022   28    34  weekend              1   

   day_of_week  store_id  quantity  rolling_7day_avg  rolling_14day_avg  \
0            0         1       0.0               0.0                0.0   
1            0         1       0.0               0.0                0.0   
2            0         1       0.0               0.0                0.0   
3            0         1       0.0               0.0                0.0   
4            0         1       0.0               0.0                0.0   

   rolling_30day_avg  
0                0.0  
1                0.0  
2          

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# Example
# combined_sales[(combined_sales['store_id'] == 1) & (combined_sales['item_id'] == 'b0d24502fb66')].head(10).T

In [5]:
df[(df['store_id'] == 1) & (df['item_id'] == '00c76368f791')].head(10).T

,0
date,2022-08-28 00:00:00
item_id,00c76368f791
month,8
year,2022
day,28
week,34
day_type,weekend
sample_weight,1
day_of_week,0
store_id,1


In [ ]:
test = db.sql("""
    select 
        a.row_id,
        a.item_id,
        a.store_id,
        extract(month from a.date) as month,
        extract(year from a.date) as year,
        extract(day from a.date) as day,
        extract(week from a.date) as week,
        extract(dayofweek from a.date) as day_of_week,
        coalesce(avg(b.rolling_7day_avg), 0) as rolling_7day_avg,
        coalesce(avg(b.rolling_14day_avg), 0) as rolling_14day_avg,
        coalesce(avg(b.rolling_30day_avg), 0) as rolling_30day_avg
    from test as a 
    left join combined_sales as b  
        on a.item_id = b.item_id
        and a.store_id = b.store_id
        and b.week in (38,39)
        and b.year = 2024
    group by 1,2,3,4,5,6,7,8 
""").df()
test.head()

In [6]:
import duckdb
import pandas as pd

# Cargar datos originales
sales = pd.read_csv('data/sales.csv', parse_dates=['date'])
online = pd.read_csv('data/online.csv', parse_dates=['date'])
test_df = pd.read_csv('data/test.csv', parse_dates=['date'], sep=';')

# Conectar a la base DuckDB en disco
con = duckdb.connect("data/mydb.duckdb")

# Registrar DataFrames como tablas temporales
con.register("sales", sales)
con.register("online", online)
con.register("test", test_df)

# (IMPORTANTE: aquí asumimos que la tabla combined_sales ya fue creada en disco con el script anterior)

# Crear tabla persistente para test
con.execute("""
    CREATE OR REPLACE TABLE test_features AS
    select 
        a.row_id,
        a.item_id,
        a.store_id,
        extract(month from a.date) as month,
        extract(year from a.date) as year,
        extract(day from a.date) as day,
        extract(week from a.date) as week,
        extract(dayofweek from a.date) as day_of_week,
        coalesce(avg(b.rolling_7day_avg), 0) as rolling_7day_avg,
        coalesce(avg(b.rolling_14day_avg), 0) as rolling_14day_avg,
        coalesce(avg(b.rolling_30day_avg), 0) as rolling_30day_avg
    from test as a 
    left join combined_sales as b  
        on a.item_id = b.item_id
        and a.store_id = b.store_id
        and b.week in (38,39)
        and b.year = 2024
    group by 1,2,3,4,5,6,7,8 
""")

# Consultar la tabla persistente
df_test = con.execute("SELECT * FROM test_features LIMIT 10").df()
print(df_test.head())

# Exportar a parquet si lo deseas
con.execute("COPY test_features TO 'data/test_features.parquet' (FORMAT 'parquet')")


/tmp/ipykernel_1227/3530420566.py:7: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  test_df = pd.read_csv('data/test.csv', parse_dates=['date'], sep=';')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   row_id       item_id  store_id  month  year  day  week  day_of_week  \
0  372929  9454fb3e603d         2     10  2024   26    43            6   
1  490919  ccc1d739eb10         3     10  2024   26    43            6   
2  486599  fbc35fe0e09d         3     10  2024   26    43            6   
3  110159  3460f346e8d6         1     10  2024   26    43            6   
4  669449  fee521eec807         4     10  2024   26    43            6   

   rolling_7day_avg  rolling_14day_avg  rolling_30day_avg  
0          0.246753           0.155844           0.169697  
1          1.831169           1.948052           1.830303  
2          0.389610           0.402597           0.578788  
3          3.242429           3.383305           3.106106  
4          2.103896           2.064935           1.557576  


In [7]:
features = ['item_id', 
            'store_id', 
            'day', 
            'week', 
            'day_of_week',
            'rolling_7day_avg',	
            'rolling_14day_avg',	
            'rolling_30day_avg', 
            'quantity',]

train = df[features]
train.shape


(10, 9)

In [ ]:
train = db.sql("""
    WITH quantiles AS (
        SELECT 
            item_id,
            store_id,
            day_of_week,
            percentile_cont(0.01) WITHIN GROUP (ORDER BY quantity) as q01,
            percentile_cont(0.99) WITHIN GROUP (ORDER BY quantity) as q99
        FROM train
        GROUP BY item_id, store_id, day_of_week
    )
    SELECT t.*
    FROM train t
    JOIN quantiles q 
        ON t.item_id = q.item_id 
        AND t.store_id = q.store_id
        AND t.day_of_week = q.day_of_week
    WHERE t.quantity > q.q01 
        AND t.quantity < q.q99
""").df()

train.shape


In [10]:
con.execute("""
    CREATE OR REPLACE TABLE train AS
    SELECT 
        item_id,
        store_id,
        day,
        week,
        day_of_week,
        rolling_7day_avg,	
        rolling_14day_avg,	
        rolling_30day_avg, 
        quantity
    FROM combined_sales
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
con.execute("""
    CREATE OR REPLACE TABLE train_filtered AS
    WITH quantiles AS (
        SELECT 
            item_id,
            store_id,
            day_of_week,
            percentile_cont(0.01) WITHIN GROUP (ORDER BY quantity) as q01,
            percentile_cont(0.99) WITHIN GROUP (ORDER BY quantity) as q99
        FROM train
        GROUP BY item_id, store_id, day_of_week
    )
    SELECT t.*
    FROM train t
    JOIN quantiles q 
        ON t.item_id = q.item_id 
        AND t.store_id = q.store_id
        AND t.day_of_week = q.day_of_week
    WHERE t.quantity > q.q01 
      AND t.quantity < q.q99
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [12]:
print("combined_sales:", con.execute("SELECT COUNT(*) FROM combined_sales").fetchone()[0], "filas")
print("test_features:", con.execute("SELECT COUNT(*) FROM test_features").fetchone()[0], "filas")
print("train:", con.execute("SELECT COUNT(*) FROM train").fetchone()[0], "filas")
print("train_filtered:", con.execute("SELECT COUNT(*) FROM train_filtered").fetchone()[0], "filas")

combined_sales: 44363256 filas
test_features: 883680 filas
train: 44363256 filas
train_filtered: 6562913 filas


In [15]:
predictor = TabularPredictor(label='quantity', eval_metric='root_mean_squared_error')

No path specified. Models will be saved in: "AutogluonModels/ag-20250826_053402"


In [16]:
tree_hyperparameters = {
    'GBM': {},   
    'XGB': {},   
    'RF': {},    
    'XT': {},    
    'CAT': {}    
}

In [21]:
train = con.execute("SELECT * FROM train_filtered").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [22]:
predictor.fit(train, 
              presets='medium_quality', 
              time_limit=60*15, 
              verbosity=2, 
              hyperparameters = tree_hyperparameters)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.8
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025
CPU Count:          4
Memory Avail:       1.35 GB / 5.79 GB (23.3%)
Disk Space Avail:   882.30 GB / 1006.85 GB (87.6%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "/home/maxkaizo/forecasting_2025/explore/autogluon/AutogluonModels/ag-20250826_053402"
Train Data Rows:    6562913
Train Data Columns: 8
Label Column:       quantity
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (4736.0, -150.0, 5.77516, 27.03577)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init

RuntimeError: No models were trained successfully during fit(). Inspect the log output or increase verbosity to determine why no models were fit. Alternatively, set `raise_on_no_models_fitted` to False during the fit call.

In [ ]:
print(predictor.feature_importance(train))

In [ ]:
features_test = ['item_id', 
            'store_id', 
            'day', 
            'week', 
            'day_of_week',
            'rolling_7day_avg',	
            'rolling_14day_avg',	
            'rolling_30day_avg', 
            ]

test[features_test].head()

In [ ]:
# Make predictions
test['quantity'] = predictor.predict(test[features_test])
test.head()

In [ ]:
submission = pd.read_csv('/kaggle/input/ml-zoomcamp-2024-competition/sample_submission.csv')
submission.head()

In [ ]:
# join on item_id, store_id, date
submission = db.sql("""
    select 
        a.row_id,
        b.quantity
    from submission as a
    left join test as b
        on a.row_id = b.row_id
""").df()
submission.head(), submission.shape

In [ ]:
# Create submission file
submission.to_csv('submission_autogluon_best_recent_min_with_rolling_averages.csv', index=False)

In [ ]:
submission.info()